In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Dataset

### Get all non-working days

In [2]:
HOLIDAYS = pd.to_datetime([#'2020-03-08', 
                           '2020-03-25',
                           '2020-03-30',
                           '2020-03-31'])
HOLIDAYS.to_numpy(dtype='str')

array(['2020-03-25T00:00:00.000000000', '2020-03-30T00:00:00.000000000',
       '2020-03-31T00:00:00.000000000'], dtype='<U48')

In [3]:
SUNDAYS = pd.date_range(start='2020-3-1', end='2020-4-7', 
                         freq='W-SUN')
SUNDAYS.to_numpy(dtype='str')

array(['2020-03-01T00:00:00.000000000', '2020-03-08T00:00:00.000000000',
       '2020-03-15T00:00:00.000000000', '2020-03-22T00:00:00.000000000',
       '2020-03-29T00:00:00.000000000', '2020-04-05T00:00:00.000000000'],
      dtype='<U48')

In [4]:
NON_WORKING_DAYS = pd.to_datetime(['2020-03-01',
                                   '2020-03-08',
                                   '2020-03-15',
                                   '2020-03-22',
                                   '2020-03-25',
                                   '2020-03-29',
                                   '2020-03-30',
                                   '2020-03-31',
                                   '2020-04-05',])
NON_WORKING_DAYS

DatetimeIndex(['2020-03-01', '2020-03-08', '2020-03-15', '2020-03-22',
               '2020-03-25', '2020-03-29', '2020-03-30', '2020-03-31',
               '2020-04-05'],
              dtype='datetime64[ns]', freq=None)

### Load dataset

In [5]:
df = pd.read_csv('delivery_orders_march.csv')
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [6]:
sla = np.array([[3, 5, 7, 7],
                [5, 5, 7, 7],
                [7, 7, 7, 7],
                [7, 7, 7, 7]])
sla

array([[3, 5, 7, 7],
       [5, 5, 7, 7],
       [7, 7, 7, 7],
       [7, 7, 7, 7]])

In [7]:
place_to_idx = {'manila':0,
                'luzon':1,
                'visayas':2,
                'mindanao':3}
place_to_idx

{'manila': 0, 'luzon': 1, 'visayas': 2, 'mindanao': 3}

## Convert date format

In [8]:
df['pick'] = pd.to_datetime(df['pick'], unit='s').dt.date
df['1st_deliver_attempt'] = pd.to_datetime(df['1st_deliver_attempt'], unit='s').dt.date
df['2nd_deliver_attempt'] = pd.to_datetime(df['2nd_deliver_attempt'], unit='s').dt.date
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04,2020-03-06,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05,2020-03-07,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04,2020-03-06,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


### Get date ranges

In [9]:
df['pick'].min(), df['pick'].max()

(datetime.date(2020, 3, 1), datetime.date(2020, 4, 6))

In [10]:
df['1st_deliver_attempt'].min(), df['1st_deliver_attempt'].max()

(datetime.date(2020, 3, 2), datetime.date(2020, 4, 7))

In [11]:
df[~df['2nd_deliver_attempt'].isnull()]['2nd_deliver_attempt'].min(), df[~df['2nd_deliver_attempt'].isnull()]['2nd_deliver_attempt'].max()

(datetime.date(2020, 3, 3), datetime.date(2020, 4, 7))

### Find anomalous data

In [12]:
df[(df['pick'] > df['1st_deliver_attempt']) |
   (df['pick'] > df['2nd_deliver_attempt']) |
   (df['1st_deliver_attempt'] > df['2nd_deliver_attempt'])]

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress


### Calc delivery dates

In [13]:
%%time
df['days_pick_to_1st_attempt'] = (df['1st_deliver_attempt'] - df['pick']).dt.days

holidays_pick_to_1st_attempt = np.zeros(len(df))
for holiday in NON_WORKING_DAYS:
    holidays_pick_to_1st_attempt += ((df['pick'] <= holiday) & (holiday <= df['1st_deliver_attempt'])).to_numpy()*1
df['holidays_pick_to_1st_attempt'] = holidays_pick_to_1st_attempt.astype('int')

df['working_days_pick_to_1st_attempt'] = df['days_pick_to_1st_attempt'] - df['holidays_pick_to_1st_attempt']
df.head()

Wall time: 23.1 s


,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,days_pick_to_1st_attempt,holidays_pick_to_1st_attempt,working_days_pick_to_1st_attempt
0,2215676524,2020-03-02,2020-03-05,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,3,0,3
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",2,0,2
2,2220979489,2020-03-04,2020-03-06,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",2,0,2
3,2221066352,2020-03-05,2020-03-07,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",2,0,2
4,2222478803,2020-03-04,2020-03-06,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,2,0,2


In [14]:
%%time
df['days_1st_to_2nd_attempt'] = (df['2nd_deliver_attempt'] - df['1st_deliver_attempt']).dt.days
df.head()

holidays_1st_to_2nd_attempt = np.zeros(len(df))
for holiday in NON_WORKING_DAYS:
    holidays_1st_to_2nd_attempt += ((df['1st_deliver_attempt'] <= holiday) & (holiday <= df['2nd_deliver_attempt'])).to_numpy()*1
df['holidays_1st_to_2nd_attempt'] = holidays_1st_to_2nd_attempt.astype('int')
df.head()

df['working_days_1st_to_2nd_attempt'] = (df['days_1st_to_2nd_attempt'] - df['holidays_1st_to_2nd_attempt'])
df.head()

Wall time: 14.5 s


,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,days_pick_to_1st_attempt,holidays_pick_to_1st_attempt,working_days_pick_to_1st_attempt,days_1st_to_2nd_attempt,holidays_1st_to_2nd_attempt,working_days_1st_to_2nd_attempt
0,2215676524,2020-03-02,2020-03-05,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,3,0,3,NaN,0,NaN
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",2,0,2,4.0,1,3.0
2,2220979489,2020-03-04,2020-03-06,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",2,0,2,NaN,0,NaN
3,2221066352,2020-03-05,2020-03-07,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",2,0,2,NaN,0,NaN
4,2222478803,2020-03-04,2020-03-06,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,2,0,2,NaN,0,NaN


In [15]:
%%time
df['origin'] = df['selleraddress'].apply(lambda x : place_to_idx[x.split()[-1].lower()])
#df['origin'] = 0
df['destination'] = df['buyeraddress'].apply(lambda x : place_to_idx[x.split()[-1].lower()])

Wall time: 6.86 s


In [16]:
%%time
time_limits = [sla[df['origin'].iloc[i]][df['destination'].iloc[i]] for i in range(len(df))]
df['time_limit'] = time_limits

Wall time: 1min 6s


In [17]:
%%time
df['is_late'] = ((df['working_days_pick_to_1st_attempt'] > df['time_limit']) | 
                 ((~pd.isnull(df['2nd_deliver_attempt'])) &
                  (df['working_days_1st_to_2nd_attempt'] > 3)
                 )
                )*1
df.head()

Wall time: 105 ms


,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,days_pick_to_1st_attempt,holidays_pick_to_1st_attempt,working_days_pick_to_1st_attempt,days_1st_to_2nd_attempt,holidays_1st_to_2nd_attempt,working_days_1st_to_2nd_attempt,origin,destination,time_limit,is_late
0,2215676524,2020-03-02,2020-03-05,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,3,0,3,NaN,0,NaN,0,0,3,0
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",2,0,2,4.0,1,3.0,0,0,3,0
2,2220979489,2020-03-04,2020-03-06,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",2,0,2,NaN,0,NaN,0,0,3,0
3,2221066352,2020-03-05,2020-03-07,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",2,0,2,NaN,0,NaN,0,0,3,0
4,2222478803,2020-03-04,2020-03-06,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,2,0,2,NaN,0,NaN,0,1,5,0


In [18]:
df_submission = df[['orderid', 'is_late']]
df_submission.head()

,orderid,is_late
0,2215676524,0
1,2219624609,0
2,2220979489,0
3,2221066352,0
4,2222478803,0


In [19]:
df_submission.sum()

orderid   -3711286817636307621
is_late                 762910
dtype: int64

In [20]:
# 764670
# 764511
# 762910
# 762910 all manila
# 763078 fixed  0.99450
# 752768
# 763184
# all leq:   762910
# leq, less: 763078

In [21]:
df_submission.to_csv('submission_14.csv', index=False)

In [22]:
len(df_submission)

3176313